In [41]:
using JuMP
using Gurobi
using Random
using Distributions

In [42]:
N=6 #No. of nodes (1 supplier + 5 DC's)

K=3 #No. of vehicles

Dk=9000 #Capacity of vehicles increased by 10 times

T=8 #Periods

h=0.025 #Holding cost/unit at each DC

C=[50000 5020 4880 2200 4860 7420] #Capacity limit for (1 supplier + 5 DC's) increased by 10 times

d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
] #Demand for each day and (1 supplier + 5 DC's)

c=[
0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0
] #Transportation cost

std=[
0 0 0 0 0 0 0 0;
11.87 11.87 11.87 11.87 11.87 14.69 14.69 11.87;
10.91 13.89 13.89 13.89 13.89 14.14 11.22 10.91;
11.27 11.27 11.27 11.27 11.27 12.86 12.86 11.27;
13.19 16.03 16.03 16.03 16.03 15.05 11.98 13.19;
13.04 14.00 14.00 14.00 14.00 11.08 9.84 13.04
]# Standard deviation of Demand


I0=[0 98.8 153.9 42.4 23.4 85.7] #Initial inventory
alpha=0.79 # Critical fractile
za=quantile.(Normal(0,1),alpha) #Safety factor
d_new = d + za.*std; #Demand to be satisfied

In [43]:
model=Model(Gurobi.Optimizer)
@variable(model, x[i=1:N,j=1:N,k=1:K,t=1:T], Bin); #If truck k leaves DC i and goes to DC j
@variable(model, y[i=1:N,k=1:K,t=1:T], Bin); #If truck k arrives at DC i
@variable(model, z[j=1:N,k=1:K,t=1:T] >= 0); #Load of truck k when arriving at node j
@variable(model, I[i=1:N,t=0:T] >= 0); #Inventory level at node i till end of period t
@variable(model, q[i=1:N,k=1:K,t=1:T] >= 0); #Quantity delivered to customer i by truck k at start of period t

@objective(model, Min, sum(h*I[i,t] for i=2:N, t=1:T) + sum(c[i,j]*x[i,j,k,t]  for i=1:N, j=1:N, k=1:K, t=1:T))

@constraint(model, [i=2:N], I[i,0] == I0[i]) #Initializing inventory level

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) == I[i,t] + d_new[i,t]); #Balance constraint for each DC

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) <= C[i]); #Capacity constraint for each DC

@constraint(model, [i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); #Constraint which ensures y is 1 when they receive a quantity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i=2:N) <= Dk*y[1,k,t]); #If truck delivers from supplier, it has to activate the supplier

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(i == h ? 0 : x[i,h,k,t] for i = 1:N) == y[h,k,t]); #If truck visits DC

@constraint(model, [h=1:N,k=1:K,t=1:T], sum(j == h ? 0 : x[h,j,k,t] for j = 1:N) == y[h,k,t]); #If truck leaves DC

@constraint(model, [i=2:N,j=2:N,k=1:K,t=1:T], z[i,k,t] - d_new[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d_new)); #To avoid subtours automatically



Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-06


In [44]:
optimize!(model)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0x1b992ef6
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 7e+03]
Found heuristic solution: objective 10487.427976
Presolve removed 160 rows and 213 columns
Presolve time: 0.01s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 3.160364e+02, 642 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  316.03639    0   95 10487.4280  316.03639  97.0%     -    0s
H    0     0          

In [45]:

if termination_status(model) == MOI.OPTIMAL
   println("RESULTS:")
   println("Objective = $(objective_value(model))")
else
   println("No solution")
end

println();
    for k=1:K
        has_route = false;
        print("Truck:$(value(k))\n")
        for t=1:T
            print("Day:$(value(t))\n")
            for i=1:N
                for j=1:N
                    if (value(x[i,j,k,t]) == 1)
                        print(j,"->",i,"\n")
                        print("$(value(q[i,k,t]))\n")
                        has_route = true;
                    end
                end
            end
        end
        if has_route
            println();
        end
    end

for i=1:N
    for k=1:K
        for t=1:T
            if (value(q[i,k,t]) > 0)
                println(" Node ",i," Truck ",k," Day ", t," Delivery ",value(q[i,k,t]))
            end
        end
    end
end


for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t]))
    end
end


RESULTS:
Objective = 1919.262185566159

Truck:1
Day:1
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8
Truck:2
Day:1
4->1
0.0
6->2
609.1259774498812
2->3
539.3517189187661
5->4
570.3713591966828
3->5
1084.378689162052
1->6
840.161388442674
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8

Truck:3
Day:1
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8
 Node 2 Truck 1 Day 5 Delivery 7.23048287909478e-11
 Node 2 Truck 2 Day 1 Delivery 609.1259774498812
 Node 2 Truck 2 Day 2 Delivery 3.808509063674137e-11
 Node 2 Truck 3 Day 4 Delivery 4.3428372009657323e-11
 Node 3 Truck 2 Day 1 Delivery 539.3517189187661
 Node 4 Truck 2 Day 1 Delivery 570.3713591966828
 Node 5 Truck 2 Day 1 Delivery 1084.378689162052
 Node 5 Truck 3 Day 7 Delivery 1.5631940186722204e-12
 Node 6 Truck 1 Day 5 Delivery 2.1884716261411086e-11
 Node 6 Truck 1 Day 7 Delivery 7.345590802287916e-11
 Node 6 Truck 2 Day 1 Delivery 840.161388442674
 Node 6 Truck 3 Day 2 Delivery 1.0157257467663571e-10
 Node 6 Truck 3 Day 3 Delivery 2.8194335754960775e

In [46]:
#Truck Capacity utilization
for k=1:K
    println("Truck ",k)
    for t=1:T
        println("on day ",t," utilizes ", value(sum(q[i,k,t] for i=1:N))/value(Dk))
    end
end

Truck 1
on day 1 utilizes 0.0
on day 2 utilizes 0.0
on day 3 utilizes 0.0
on day 4 utilizes 0.0
on day 5 utilizes 1.0465505005817654e-14
on day 6 utilizes 0.0
on day 7 utilizes 8.161767558097685e-15
on day 8 utilizes 0.0
Truck 2
on day 1 utilizes 0.4048210147966729
on day 2 utilizes 4.2316767374157074e-15
on day 3 utilizes 0.0
on day 4 utilizes 0.0
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 0.0
on day 8 utilizes 0.0
Truck 3
on day 1 utilizes 0.0
on day 2 utilizes 1.1285841630737302e-14
on day 3 utilizes 3.1327039727734195e-15
on day 4 utilizes 1.028865881380625e-14
on day 5 utilizes 0.0
on day 6 utilizes 0.0
on day 7 utilizes 1.736882242969134e-16
on day 8 utilizes 0.0


In [47]:
#Capacity utilization of the DCs
for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t])/value(C[i]))
    end
end


Day1
Node 2 Inventory Level = 0.1239947723601145
Node 3 Inventory Level = 0.12736755391676172
Node 4 Inventory Level = 0.24385590533763052
Node 5 Inventory Level = 0.20469999854195092
Node 6 Inventory Level = 0.10888755463363291
Day2
Node 2 Inventory Level = 0.10696843367444349
Node 3 Inventory Level = 0.10564599835916266
Node 4 Inventory Level = 0.20917937467676886
Node 5 Inventory Level = 0.1699413704370739
Node 6 Inventory Level = 0.08983231238858932
Day3
Node 2 Inventory Level = 0.08994209498876489
Node 3 Inventory Level = 0.08392444280156361
Node 4 Inventory Level = 0.17450284401590724
Node 5 Inventory Level = 0.13518274233219688
Node 6 Inventory Level = 0.07077707014354524
Day4
Node 2 Inventory Level = 0.07291575630309495
Node 3 Inventory Level = 0.062202887243964555
Node 4 Inventory Level = 0.1398263133550456
Node 5 Inventory Level = 0.10042411422731984
Node 6 Inventory Level = 0.05172182789850397
Day5
Node 2 Inventory Level = 0.05588941761743076
Node 3 Inventory Level = 0.04048

In [48]:
Random.seed!(0);
counter=zeros(6,8)
#probability=zeros(6,8)
for r = 1:10000
    actual_demand = d + randn(6,8).*std;
    for i=1:N
        new_IL=I0[i];
        for t=1:T
            delivered=0;
            for k=1:K
                delivered = delivered + value(q[i,k,t]);
            end
            new_IL=new_IL+delivered-value(actual_demand[i,t]);
            fillrate=(max(value(actual_demand[i,t])+min(new_IL,0),0))/value(actual_demand[i,t]);
            if fillrate>0.95
                counter[i,t]=counter[i,t] + 1
            end
        end
    end
end
probability=counter/10000

6×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9917
 1.0  1.0  1.0  1.0  1.0  1.0  0.9996  0.9908
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9931
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9923
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9927

In [49]:
probability # Probability of meeting the fillrate target

6×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9917
 1.0  1.0  1.0  1.0  1.0  1.0  0.9996  0.9908
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9931
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9923
 1.0  1.0  1.0  1.0  1.0  1.0  1.0     0.9927

In [50]:
safety_stock=za.*std # Safety stock

6×8 Matrix{Float64}:
  0.0       0.0       0.0       0.0      …   0.0       0.0       0.0
  9.57222   9.57222   9.57222   9.57222     11.8463   11.8463    9.57222
  8.79806  11.2012   11.2012   11.2012      11.4028    9.04805   8.79806
  9.08837   9.08837   9.08837   9.08837     10.3706   10.3706    9.08837
 10.6367   12.9269   12.9269   12.9269      12.1366    9.66093  10.6367
 10.5157   11.2899   11.2899   11.2899   …   8.93515   7.93519  10.5157

In [55]:
using DelimitedFiles

In [56]:
writedlm( "3.2c.csv",  safety_stock, ',')